# Determination of  ship’s effective power by implementing neural networks.

In this notebook, we have implemented neural networks to determine ship effective power at operating conditions. This notebook is a modified version of one of the applications discussed in the technical paper published in Naval Engineers Journal wherein they have have more attributes present in their dataset which isn't publicly available. Therefore, we have implemented this model with real-world data that was scraped from [Fleetmon](https://www.fleetmon.com/) which contains most of the attributes which were present in the original dataset used in the paper.

Link : [Application of Artificial Intelligence Methods to Preliminary Design of Ships and Ship Performance Optimization](https://www.researchgate.net/profile/Tomasz-Abramowski/publication/259361068_Application_of_Artificial_Intelligence_Methods_to_Preliminary_Design_of_Ships_and_Ship_Performance_Optimization/links/5ac5f1d30f7e9b1067d5715f/Application-of-Artificial-Intelligence-Methods-to-Preliminary-Design-of-Ships-and-Ship-Performance-Optimization.pdf) 


###Imports

In [129]:
from google.colab import drive

drive.mount('/content/gdrive')  #mouting google drive files into colab

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [130]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


###Loading Dataset

In [131]:
df = pd.read_csv('/content/gdrive/MyDrive/semi_final_dataset.csv', encoding = 'unicode_escape')  # load dataset
df.head() #1st five rows

,Sr No,Vessel_Name,Length(m),Width(m),Draught Avg(m),Speed Avg(knots),Speed Max(knots),Deadweight(tonnes),Gross Tonnage(tonnes),Total Power (kW),Engine,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,0,MAREN MAERSK,399,59,12,15.1,23.2,196000,194849,23310.0,Propulsion SystemPropeller Shaft PortManufactu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,MARCHEN MAERSK,399,59,13.2,14.7,24.9,196000,194849,23310.0,Propulsion SystemPropeller Shaft StarboardManu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,MATHILDE MAERSK,399,59,13.8,15.0,24.1,196000,194849,23310.0,Propulsion SystemPropeller Shaft PortManufactu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,METTE MAERSK,399,59,13.9,14.9,24.2,196000,194849,23310.0,Propulsion SystemPropeller Shaft PortManufactu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,MARGRETHE MAERSK,399,59,13.7,14.6,24.2,194916,194849,46620.0,Machinery Overview2 oil engines driving 2 FP p...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's have a look at this data to see what we are working with.

In [132]:
df.shape  #Returning a tuple conataning dimension of df in the form (rows, columns)

(401, 24)

In [133]:
df.dtypes #returning a series with the data type of each column of df

Sr No                      int64
Vessel_Name               object
Length(m)                  int64
Width(m)                   int64
Draught Avg(m)            object
Speed Avg(knots)         float64
Speed Max(knots)          object
Deadweight(tonnes)         int64
Gross Tonnage(tonnes)      int64
Total Power (kW)         float64
Engine                    object
Unnamed: 11              float64
Unnamed: 12              float64
Unnamed: 13              float64
Unnamed: 14              float64
Unnamed: 15              float64
Unnamed: 16              float64
Unnamed: 17              float64
Unnamed: 18              float64
Unnamed: 19              float64
Unnamed: 20              float64
Unnamed: 21              float64
Unnamed: 22              float64
Unnamed: 23              float64
dtype: object

After looking at the columns we can see that the columns which start with name "Unnamed" aren't useful to us. Therefore we have to remove those colummns. This will be dealt with in the preprocessing section.

##Data Preprocessing
The last step before creating our model is to *preprocess* our data. 

First, we only choose columns which are useful to us from the extracted dataset.
Then we will try to create new columns which are useful for our particular case here. Finally, we will end with normalization of the whole dataset and move towards making our model.



In [134]:
df_final = df.iloc[:,2:10]    #saving colunms from index 2 to 9 from df to df_final since column 0,1 are just serial nos and column 10 to 23 are 
df_final

,Length(m),Width(m),Draught Avg(m),Speed Avg(knots),Speed Max(knots),Deadweight(tonnes),Gross Tonnage(tonnes),Total Power (kW)
0,399,59,12,15.1,23.2,196000,194849,23310.0
1,399,59,13.2,14.7,24.9,196000,194849,23310.0
2,399,59,13.8,15.0,24.1,196000,194849,23310.0
3,399,59,13.9,14.9,24.2,196000,194849,23310.0
4,399,59,13.7,14.6,24.2,194916,194849,46620.0
...,...,...,...,...,...,...,...,...
396,299,40,11.7,13.9,24.2,85823,75590,57100.0
397,243,32,7.6,12.8,23,42960,37579,28350.0
398,210,30,10.2,13.0,22.5,33411,26836,21560.0
399,202,31,9.3,14.0,21,31160,30567,16260.0


We have to remove the rows with NaN value

In [135]:
df_final.dropna(axis=0,inplace=True) #updating df_final by droping rows which contain missing values 
df_final.head()

,Length(m),Width(m),Draught Avg(m),Speed Avg(knots),Speed Max(knots),Deadweight(tonnes),Gross Tonnage(tonnes),Total Power (kW)
0,399,59,12,15.1,23.2,196000,194849,23310.0
1,399,59,13.2,14.7,24.9,196000,194849,23310.0
2,399,59,13.8,15.0,24.1,196000,194849,23310.0
3,399,59,13.9,14.9,24.2,196000,194849,23310.0
4,399,59,13.7,14.6,24.2,194916,194849,46620.0


In [136]:
df_final.shape  #Returning a tuple conataning dimension of df_final in the form (rows, columns)

(356, 8)

In [137]:
df_final.dtypes #returning a series with the data type of each column of df_final

Length(m)                  int64
Width(m)                   int64
Draught Avg(m)            object
Speed Avg(knots)         float64
Speed Max(knots)          object
Deadweight(tonnes)         int64
Gross Tonnage(tonnes)      int64
Total Power (kW)         float64
dtype: object

After looking into the type of various columns, we realize that columns such as 
*Draught Avg(m)* and *Speed Max(knots)* should be float values instead of object values. Therefore, we convert them to float values.

In [138]:
cols = ['Draught Avg(m)','Speed Max(knots)']    
df_final[cols] = df_final[cols].apply(pd.to_numeric, errors='coerce') #Converting to a numeric type, errors='coerce' will covert invalid parsing into NaN
df_final.dtypes #returning a series with the data type of each column of df_final

Length(m)                  int64
Width(m)                   int64
Draught Avg(m)           float64
Speed Avg(knots)         float64
Speed Max(knots)         float64
Deadweight(tonnes)         int64
Gross Tonnage(tonnes)      int64
Total Power (kW)         float64
dtype: object

Now we proceed to create new columns which are not derived from the heatmap of the dataset but from the knowledge of the problem that we are trying to solve. 

In [139]:
df_final['Froude Number'] = df_final['Speed Avg(knots)']/((9.81*df_final['Length(m)'])**(1/2))  #adding new column "Froude Number" using "speed avg(knots)" column to df_final
df_final['L/B'] = df_final['Length(m)']/(df_final['Width(m)'])  #adding new column "L/B" using "Length(m)" and "Width(m)" column to df_final
df_final['B/T'] = df_final['Width(m)']/(df_final['Draught Avg(m)']) #adding new column "L/B" using "Width(m" and "Draught Avg(m)" column to df_final
df_final.head()

,Length(m),Width(m),Draught Avg(m),Speed Avg(knots),Speed Max(knots),Deadweight(tonnes),Gross Tonnage(tonnes),Total Power (kW),Froude Number,L/B,B/T
0,399,59,12.0,15.1,23.2,196000,194849,23310.0,0.241355,6.762712,4.916667
1,399,59,13.2,14.7,24.9,196000,194849,23310.0,0.234961,6.762712,4.469697
2,399,59,13.8,15.0,24.1,196000,194849,23310.0,0.239756,6.762712,4.275362
3,399,59,13.9,14.9,24.2,196000,194849,23310.0,0.238158,6.762712,4.244604
4,399,59,13.7,14.6,24.2,194916,194849,46620.0,0.233363,6.762712,4.306569


In [140]:
df_final.shape  #Returning a tuple conataning dimension of df_final in the form (rows, columns)

(356, 11)

Finally, we move our target column **Total Power (kW)** to the end of the dataset

In [141]:
column_to_move = df_final.pop('Total Power (kW)') #deleting the column "Total Power (kW)" from df_final and saving it in column_to_move


# insert column with insert(location, column_name, column_value)

df_final.insert(df_final.shape[1], 'Total Power (kW)', column_to_move)  #inserting column "Total Power (kW)" at the end of df_final
df_final.head()

,Length(m),Width(m),Draught Avg(m),Speed Avg(knots),Speed Max(knots),Deadweight(tonnes),Gross Tonnage(tonnes),Froude Number,L/B,B/T,Total Power (kW)
0,399,59,12.0,15.1,23.2,196000,194849,0.241355,6.762712,4.916667,23310.0
1,399,59,13.2,14.7,24.9,196000,194849,0.234961,6.762712,4.469697,23310.0
2,399,59,13.8,15.0,24.1,196000,194849,0.239756,6.762712,4.275362,23310.0
3,399,59,13.9,14.9,24.2,196000,194849,0.238158,6.762712,4.244604,23310.0
4,399,59,13.7,14.6,24.2,194916,194849,0.233363,6.762712,4.306569,46620.0


### Clean the data

The dataset contains a few unknown values: Now we remove those entries 
before proceeding towards normalization.

In [142]:
df_final.isnull().sum().sum() #total number of "NaN" in df_final

4

In [143]:
df_final.dropna(axis=0,inplace=True)   #updating df_final by droping rows which contain missing values

In [144]:
df_final.isnull().sum().sum() #total number of "NaN" in df_final

0

Finally we store our training variables and target variable in *train_labels* and *test_label* respectively.


In [145]:
train_labels = df_final.columns[:-1]  #saving all the column except "Total Power" column in train_labels
train_labels

Index(['Length(m)', 'Width(m)', 'Draught Avg(m)', 'Speed Avg(knots)',
       'Speed Max(knots)', 'Deadweight(tonnes)', 'Gross Tonnage(tonnes)',
       'Froude Number', 'L/B', 'B/T'],
      dtype='object')

In [146]:
test_label = df_final.columns[-1]   #saving  column "Total Power" column in test_labels
test_label

'Total Power (kW)'

In [147]:
df_final.describe().transpose() #

,count,mean,std,min,25%,50%,75%,max
Length(m),354.0,304.163842,65.516308,100.000000,274.000000,304.500000,366.000000,399.000000
Width(m),354.0,41.220339,8.577395,20.000000,32.000000,42.000000,48.000000,59.000000
Draught Avg(m),354.0,11.395198,1.830820,2.000000,10.300000,11.700000,12.800000,14.800000
Speed Avg(knots),354.0,13.396045,1.061447,8.400000,12.900000,13.500000,14.000000,16.600000
Speed Max(knots),354.0,23.999435,1.974770,15.600000,23.300000,24.200000,25.100000,29.700000
Deadweight(tonnes),354.0,95539.632768,46898.937288,10600.000000,60604.000000,93620.500000,140969.250000,196000.000000
Gross Tonnage(tonnes),354.0,90841.183616,48440.130304,8656.000000,52699.000000,87757.500000,141003.000000,215860.000000
Froude Number,354.0,0.249657,0.032838,0.198534,0.230894,0.241329,0.256520,0.488491
L/B,354.0,7.393392,0.768162,1.785714,7.000000,7.345788,7.711111,9.218750
B/T,354.0,3.675010,1.470971,2.352941,3.293688,3.597753,3.907428,29.500000


### Split the data into training and test sets

Now, split the dataset into a training set and a test set. You will use the test set in the final evaluation of your models.

In [148]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( df_final[train_labels], df_final[test_label], test_size=0.2, random_state=42)
#saving random  20% of train_label to X_test
#saving random  20% of test_label to Y_test
#saving random  80% of train_label to X_train
#saving random  80% of test_label to Y_train

In [149]:
X_train.shape #Returning a tuple conataning dimension of X_train in the form (rows, columns)

(283, 10)

## Data Normalization

Now we proceed to define a normalization layer which will normalize our data while we are running the model and not normalize the dataset directly.

The `tf.keras.layers.Normalization` is a clean and simple way to add feature normalization into your model.

The first step is to create the layer:

In [150]:
normalizer = tf.keras.layers.Normalization(axis=-1)

Then, fit the state of the preprocessing layer to the data by calling `Normalization.adapt`:

In [151]:
normalizer.adapt(df_final[train_labels])

Calculating the mean:

In [152]:
print(normalizer.mean.numpy())

[[3.0416382e+02 4.1220333e+01 1.1395197e+01 1.3396046e+01 2.3999435e+01
  9.5539641e+04 9.0841164e+04 2.4965744e-01 7.3933911e+00 3.6750097e+00]]


##Building the Model
Now it's time to build the model! We are going to use a keras *sequential* model with three different layers. This model represents a feed-forward neural network (one that passes values from left to right). We'll break down each layer and its architecture below.

In [153]:
model = keras.Sequential([
    normalizer,         # input layer (1)
    keras.layers.Dense(10, activation='swish'),  # hidden layer (2)
    keras.layers.Dense(10, activation='swish'),  # hidden layer (3)
    keras.layers.Dense(10, activation='swish'),  # hidden layer (4)
    keras.layers.Dense(1) # output layer (3)
])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_4 (Normalizati (None, 10)                21        
_________________________________________________________________
dense_16 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_17 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_18 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 11        
Total params: 362
Trainable params: 341
Non-trainable params: 21
_________________________________________________________________


**Layer 1:** Our input layer is a normalization layer which provides data to the model normalized during execution. We use the normalization layer with an input shape of (10,10) to denote that our input should come in in that shape. 

**Layer 2,3,4:** These are our hidden layers. The *dense* denotes that this layer will be fully connected and each neuron from the previous layer connects to each neuron of this layer. 
Each hidden layer has  has 100 neurons and uses the swish  activation function 

`swish(x) = x * sigmoid(x)`

**Layer 5:** This is our output later and is also a dense layer. It has 1 neuron that we will look at to determine our models output.

###Compile the Model

Once the model is built, configure the training procedure using the Keras `Model.compile` method. The most important arguments to compile are the `loss` and the `optimizer`, since these define what will be optimized (`mean_absolute_error`) and how (using the `tf.keras.optimizers.Adam`).

In [154]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
              loss='mean_absolute_error',
              metrics = ['mean_squared_error']
              )

##Training the Model

Now it's finally time to train the model. Since we've already done all the work on our data this step is as easy as calling a single method.

Having configured the training configured, use Keras `Model.fit` to execute the training for 100 epochs:

In [155]:
model.fit(X_train, y_train, batch_size= 128 ,epochs=100,validation_split = 0.1)  # we pass the data, labels and epochs

Epoch 1/100
2/2 [==============================] - 1s 186ms/step - loss: 51654.1367 - mean_squared_error: 3096751872.0000 - val_loss: 52500.9180 - val_mean_squared_error: 3213523200.0000
Epoch 2/100
2/2 [==============================] - 0s 21ms/step - loss: 51646.1250 - mean_squared_error: 3096205824.0000 - val_loss: 52475.9844 - val_mean_squared_error: 3211820032.0000
Epoch 3/100
2/2 [==============================] - 0s 21ms/step - loss: 51612.0938 - mean_squared_error: 3093620736.0000 - val_loss: 52400.4336 - val_mean_squared_error: 3206633216.0000
Epoch 4/100
2/2 [==============================] - 0s 20ms/step - loss: 51520.8047 - mean_squared_error: 3086178560.0000 - val_loss: 52219.3203 - val_mean_squared_error: 3194209024.0000
Epoch 5/100
2/2 [==============================] - 0s 21ms/step - loss: 51296.7734 - mean_squared_error: 3069550336.0000 - val_loss: 51840.0508 - val_mean_squared_error: 3168151552.0000
Epoch 6/100
2/2 [==============================] - 0s 23ms/step - los

##Evaluating the Model
Now it's time to test/evaluate the model. We can do this quite easily using another builtin method from keras.

The *verbose* argument is defined from the keras documentation as:
"verbose: 0 or 1. Verbosity mode. 0 = silent, 1 = progress bar."


In [156]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=1) 

3/3 [==============================] - 0s 3ms/step - loss: 7508.5244 - mean_squared_error: 107133096.0000


##Making Predictions
To make predictions we simply need to pass an array of data in the form we've specified in the input layer to ```.predict()``` method.

In [157]:
predictions = model.predict(X_test)

Converting dataframe to NumPy array for combining both the predicted and real data

In [158]:
real = y_test.to_numpy()

Reshaping the test data to the same of predicted data

In [159]:
real= real.reshape(predictions.shape)

Combining both the predicted values and the actual values for a side-by-side comparison

In [160]:
results = pd.DataFrame(predictions,columns = ['Predicted'])
results['Real'] = real
results

,Predicted,Real
0,58618.039062,48600.0
1,60504.183594,72240.0
2,19432.945312,10780.0
3,55175.824219,41184.0
4,65485.500000,72240.0
...,...,...
66,9807.993164,12600.0
67,58448.828125,62920.0
68,47132.640625,39415.0
69,68847.812500,72240.0


# Remarks

We have tried to implement the approach followed in the paper with modifications based on the constraints we faced. But as you can see the model is far from predicting accurate values. 

There are some specific factors that contribute to this:

1. The dataset used in the paper isn't accessible or present in the internet since those readings were generated in labs. Therefore recreating the same results isn't possible. Therefore, we had to scrape data from a website which also doesn't provide us with a big dataset.
2. The real world data collected isn't sufficient enough in terms of attributes needed for the model (i.e., variables such as block coefficient  `Cb` and Waterplane Area Coefficient  `Cw` aren't available to use and can't be calculated with the given dataset)
3. Removal of outliers hasn't been done because of the small dataset that is available to us.

With better quality and quantity of data, we can however achieve better results than that of achived here. This just shows how such simple models can be used for solving complex problems in real-time and how we could exploit its advantage for development of the field.